# Sentinel Hub Feature Info Service (FIS)

A statistical summary of satellite indices can be requested via the FIS feature. Examples of extracting time-series statistics data of Sentinel-2 and Sentinel-5P indices are provided.<br>

**Note: Cloud mask values have been included along with the obtained statistics values of Sentinel-2 indices in the following examples.**

In [82]:
# import libraries
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from shapely.geometry import Polygon
import geopandas as gpd

from sentinelhub import FisRequest, BBox, Geometry, CRS, WcsRequest, CustomUrlParam, \
    DataCollection, HistogramType, bbox_to_dimensions
from sentinelhub.time_utils import iso_to_datetime

In [83]:
# sentinel hub configurations
from sentinelhub import SHConfig


INSTANCE_ID = ''  # In case you put instance ID into configuration file you can leave this unchanged

if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None

## Parameters of FIS request


* `layer` -   name of the layer defined in Sentinel Hub Configurator.
* `geometry_list` - list of geometry objects (BBox or Geometry), statistics will be calculated for each of them separately
* `time` - statistics will be calculated for each acquisition in the give time interval separately 
* `resolution` - spatial resolution on which to calculate statistics 
* `data_folder` - optional parameter for specifying location where the data should be saved locally 
* `bins` - The number of bins (a positive integer) in the histogram. When this parameter is absent, no histogram is computed.
* `histogram_type` - ways of dividing values into bins currently supported: EQUIDISTANT, EQUALFREQUENCY, or STREAMING

In [84]:
def fis_data_to_dataframe(fis_data):
    """ Creates a DataFrame from list of FIS responses
    """
    COLUMNS = ['channel', 'date', 'min', 'max', 'mean', 'stDev']
    data = []

    for fis_response in fis_data:
        for channel, channel_stats in fis_response.items():
            for stat in channel_stats:
                row = [int(channel[1:]), iso_to_datetime(stat['date'])]

                for column in COLUMNS[2:]:
                    row.append(stat['basicStats'][column])

                data.append(row)

    return pd.DataFrame(data, columns=COLUMNS).sort_values(['channel', 'date'])

In [85]:
# NDVI request
def ndvi_request(city_shp, start_date, end_date, city_name):
    geometry = Geometry(city_shp.geometry.values[0], crs=CRS.WGS84)
    time_interval = (start_date, end_date)
    ndvi_stats_fis = FisRequest(
        data_collection=DataCollection.SENTINEL2_L2A,
        layer='NDVI',
        geometry_list=[geometry],
        time=time_interval,
        resolution='100m',
        bins=20, # remove this if distribution values not needed
        histogram_type=HistogramType.EQUIDISTANT, # remove this if distribution values not needed
        config=config
    )
    ndvi_stats = ndvi_stats_fis.get_data()  
    ndvi_stats_df = clean_df(ndvi_stats)
    ndvi_stats_df.to_csv(f'ndvi-{city_name}.csv')
    return ndvi_stats_df

# NDBI request
def ndbi_request(city_shp, start_date, end_date, city_name):
    geometry = Geometry(city_shp.geometry.values[0], crs=CRS.WGS84)
    time_interval = (start_date, end_date)
    ndbi_stats_fis = FisRequest(
        data_collection=DataCollection.SENTINEL2_L2A,
        layer='NDBI',
        geometry_list=[geometry],
        time=time_interval,
        resolution='100m',
        bins=20, # remove this if distribution values not needed
        histogram_type=HistogramType.EQUIDISTANT, # remove this if distribution values not needed
        config=config
    )
    ndbi_stats = ndbi_stats_fis.get_data()  
    ndbi_stats_df = clean_df(ndbi_stats)
    ndbi_stats_df.to_csv(f'ndbi-{city_name}.csv')
    return ndbi_stats_df

# NO2 request
def no_request(city_shp, start_date, end_date, city_name):
    geometry = Geometry(city_shp.geometry.values[0], crs=CRS.WGS84)
    time_interval = (start_date, end_date)
    no_stats_fis = FisRequest(
        data_collection=DataCollection.SENTINEL5P,
        layer='NO2',
        geometry_list=[geometry],
        time=time_interval,
        resolution='100m',
        config=config
    )
    no_stats = no_stats_fis.get_data()  
    no_stats_df = clean_df(no_stats)
    no_stats_df.to_csv(f'no2-{city_name}.csv')
    return no_stats_df

In [86]:
# convert to clean df ready for analysis
def clean_df (json_stats):
    raw_df_stats = fis_data_to_dataframe(json_stats).reset_index(drop=True)
    index_df = raw_df_stats[raw_df_stats.channel ==0]
    cloud_df = raw_df_stats[raw_df_stats.channel ==1]
    cloud_df = cloud_df.drop(['channel'], axis=1).add_suffix('_clm')
    merged_df = pd.merge(index_df, cloud_df, left_on ='date', right_on = 'date_clm', how='outer')
    clean_merged_df = merged_df.drop(['channel', 'date_clm'], axis=1)
    return clean_merged_df

In [87]:
# load shapefiles of new wave cities

# sta rosa city
sta_rosa = gpd.read_file('nw_cities_shp/Sta Rosa.shp')

# lipa city
lipa = gpd.read_file('nw_cities_shp/Lipa City.shp')

# dasmarinas city
dasmarinas = gpd.read_file('nw_cities_shp/Dasmarinas.shp')

# naga city
naga = gpd.read_file('nw_cities_shp/Naga City.shp')

# baguio city
baguio = gpd.read_file('nw_cities_shp/Baguio.shp')

# CDO city
cdo = gpd.read_file('nw_cities_shp/Cagayan de Oro.shp')

## Sta Rosa City

In [88]:
# ndbic
ndbi_request(sta_rosa,'2021-9-01', '2021-9-30','sta-rosa')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-04,-1.000000,0.395950,-0.168596,0.116368,0.0,1.0,0.965938,0.181389
1,2021-09-09,-1.000000,1.000000,-0.128280,0.201690,0.0,1.0,0.059820,0.237154
2,2021-09-14,-0.516005,1.000000,-0.110521,0.200127,0.0,1.0,0.037112,0.189037
3,2021-09-19,-0.718265,-0.560652,-0.635470,0.039686,1.0,1.0,1.000000,0.000000
4,2021-09-24,-1.000000,1.000000,-0.096501,0.200692,0.0,1.0,0.008304,0.090745
5,2021-09-29,-0.460669,0.246645,-0.129095,0.091238,0.0,1.0,0.999322,0.026027


In [17]:
# ndvi
ndvi_request(sta_rosa,'2021-9-01', '2021-9-30','sta-rosa')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-04,-1.000000,1.000000,0.158267,0.271893,0.0,1.0,0.965938,0.181389
1,2021-09-09,-1.000000,1.000000,0.501518,0.317544,0.0,1.0,0.059820,0.237154
2,2021-09-14,-1.000000,1.000000,0.523672,0.334050,0.0,1.0,0.037112,0.189037
3,2021-09-19,0.000883,0.038032,0.016582,0.006703,1.0,1.0,1.000000,0.000000
4,2021-09-24,-1.000000,1.000000,0.529691,0.328300,0.0,1.0,0.008304,0.090745
5,2021-09-29,-0.466097,0.840690,0.179279,0.201828,0.0,1.0,0.999322,0.026027


In [62]:
# no2
no_request(sta_rosa,'2021-9-01', '2021-9-30','sta-rosa')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,0.000037,0.000049,0.00004,0.000005,NaN,NaN,NaN,NaN
1,2021-09-02,0.000013,0.000033,0.000032,0.000004,NaN,NaN,NaN,NaN
2,2021-09-03,0.000041,0.000087,0.000086,0.000008,NaN,NaN,NaN,NaN
3,2021-09-04,0.00003,0.000071,0.000041,0.000013,NaN,NaN,NaN,NaN
4,2021-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-09-06,0.000029,0.000048,0.000033,0.000003,NaN,NaN,NaN,NaN
6,2021-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Lipa City 

In [89]:
# ndvi
ndbi_request(lipa,'2021-9-0c1', '2021-9-30','lipa')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-04,-1.000000,1.000000,-0.227747,0.134784,0.0,1.0,0.741569,0.437772
1,2021-09-09,-1.000000,1.000000,-0.208510,0.138715,0.0,1.0,0.828171,0.377232
2,2021-09-14,-1.000000,1.000000,-0.220993,0.172052,0.0,1.0,0.325064,0.468399
3,2021-09-19,-0.760856,-0.558421,-0.647083,0.029149,1.0,1.0,1.000000,0.000000
4,2021-09-24,-1.000000,1.000000,-0.195425,0.162615,0.0,1.0,0.446510,0.497131
5,2021-09-29,-0.544721,0.169137,-0.276735,0.079326,1.0,1.0,1.000000,0.000000


In [18]:
ndvi_request(lipa,'2021-9-01', '2021-9-30','lipa')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-04,-1.000000,1.000000,0.445838,0.310395,0.0,1.0,0.741569,0.437772
1,2021-09-09,-1.000000,1.000000,0.345226,0.340423,0.0,1.0,0.828171,0.377232
2,2021-09-14,-1.000000,1.000000,0.613918,0.313166,0.0,1.0,0.325064,0.468399
3,2021-09-19,-0.012875,0.061279,0.015574,0.007428,1.0,1.0,1.000000,0.000000
4,2021-09-24,-1.000000,1.000000,0.501906,0.354422,0.0,1.0,0.446510,0.497131
5,2021-09-29,-0.048028,0.726702,0.336904,0.154124,1.0,1.0,1.000000,0.000000


In [61]:
no_request(lipa,'2021-9-01', '2021-9-30','lipa')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-04,0.000018,0.000044,0.000033,0.000007,NaN,NaN,NaN,NaN
4,2021-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-09-06,0.000015,0.000015,0.000015,0.0,NaN,NaN,NaN,NaN
6,2021-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-09,0.000001,0.000037,0.000022,0.000008,NaN,NaN,NaN,NaN
9,2021-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Dasmarinas City 

In [91]:
# ndvi
ndbi_request(dasmarinas,'2021-9-01', '2021-9-30','dasmarinas')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-04,-0.662432,1.000000,-0.163535,0.149406,0.0,1.0,0.784589,0.411107
1,2021-09-09,-1.000000,0.641407,-0.132734,0.128674,0.0,1.0,0.934168,0.247988
2,2021-09-14,-1.000000,1.000000,-0.136982,0.185552,0.0,1.0,0.217246,0.412371
3,2021-09-19,-0.707590,-0.584459,-0.643283,0.022080,1.0,1.0,1.000000,0.000000
4,2021-09-24,-1.000000,1.000000,-0.117952,0.194634,0.0,1.0,0.084287,0.277817
5,2021-09-29,-0.506935,0.330357,-0.144721,0.103303,1.0,1.0,1.000000,0.000000


In [19]:
ndvi_request(dasmarinas,'2021-9-01', '2021-9-30','dasmarinas')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-04,-1.000000,1.000000,0.397376,0.307945,0.0,1.0,0.784589,0.411107
1,2021-09-09,-0.715496,1.000000,0.281581,0.281444,0.0,1.0,0.934168,0.247988
2,2021-09-14,-1.000000,1.000000,0.548557,0.301314,0.0,1.0,0.217246,0.412371
3,2021-09-19,-0.004112,0.039941,0.019438,0.004796,1.0,1.0,1.000000,0.000000
4,2021-09-24,-1.000000,1.000000,0.578049,0.306945,0.0,1.0,0.084287,0.277817
5,2021-09-29,-0.458382,0.981876,0.353552,0.222547,1.0,1.0,1.000000,0.000000


In [64]:
no_request(dasmarinas,'2021-9-01', '2021-9-30','dasmarinas')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,0.000039,0.000039,0.000039,0.0,NaN,NaN,NaN,NaN
1,2021-09-02,0.000014,0.000041,0.000016,0.000008,NaN,NaN,NaN,NaN
2,2021-09-03,0.000019,0.000052,0.000042,0.000011,NaN,NaN,NaN,NaN
3,2021-09-04,0.000036,0.000051,0.000045,0.000003,NaN,NaN,NaN,NaN
4,2021-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-09-06,0.000025,0.000075,0.000033,0.000015,NaN,NaN,NaN,NaN
6,2021-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Naga City 

In [92]:
# ndvi
ndbi_request(naga,'2021-9-01', '2021-9-30','naga')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,-0.306436,-0.047386,-0.175040,0.045422,1.0,1.0,1.000000,0.000000
1,2021-09-06,-0.441670,0.168800,-0.178781,0.086241,1.0,1.0,1.000000,0.000000
2,2021-09-11,-0.394803,-0.074040,-0.233934,0.041265,1.0,1.0,1.000000,0.000000
3,2021-09-16,-0.540699,1.000000,-0.179885,0.065772,0.0,1.0,0.993760,0.078744
4,2021-09-21,-1.000000,1.000000,-0.155336,0.141668,0.0,1.0,0.745740,0.435444
5,2021-09-26,-0.749839,0.296067,-0.271402,0.134779,0.0,1.0,0.988121,0.108342


In [20]:
ndvi_request(naga,'2021-9-01', '2021-9-30','naga')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,0.019005,0.920455,0.116704,0.066429,1.0,1.0,1.000000,0.000000
1,2021-09-06,-0.052031,0.873513,0.338949,0.218207,1.0,1.0,1.000000,0.000000
2,2021-09-11,-0.018661,0.552433,0.231225,0.127175,1.0,1.0,1.000000,0.000000
3,2021-09-16,-0.128635,1.000000,0.252424,0.151006,0.0,1.0,0.993760,0.078744
4,2021-09-21,-1.000000,1.000000,0.370525,0.348309,0.0,1.0,0.745740,0.435444
5,2021-09-26,-0.137758,1.000000,0.152246,0.216070,0.0,1.0,0.988121,0.108342


In [65]:
no_request(naga,'2021-9-01', '2021-9-30','naga')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,0.000022,0.000026,0.000024,0.000002,NaN,NaN,NaN,NaN
1,2021-09-02,0.000016,0.000021,0.000019,0.000002,NaN,NaN,NaN,NaN
2,2021-09-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-04,0.000011,0.000015,0.000012,0.000002,NaN,NaN,NaN,NaN
4,2021-09-05,0.000016,0.000045,0.000028,0.000014,NaN,NaN,NaN,NaN
5,2021-09-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-09,0.000009,0.000022,0.000016,0.000006,NaN,NaN,NaN,NaN
9,2021-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Baguio City 

In [93]:
# ndvi
ndbi_request(baguio,'2021-9-01', '2021-9-30','baguio')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-02,-0.955752,1.000000,-0.135100,0.135332,0.0,1.0,0.672821,0.469183
1,2021-09-04,-0.569322,-0.321733,-0.495350,0.037035,1.0,1.0,1.000000,0.000000
2,2021-09-07,-0.657911,0.898990,-0.143609,0.126500,0.0,1.0,0.540253,0.498377
3,2021-09-09,-0.558935,0.576819,-0.131444,0.152451,0.0,1.0,0.083001,0.275883
4,2021-09-12,-0.644386,-0.605935,-0.622524,0.008570,1.0,1.0,1.000000,0.000000
5,2021-09-14,-1.000000,1.000000,-0.096687,0.155423,0.0,1.0,0.592315,0.491404
6,2021-09-17,-0.436193,0.404665,-0.165830,0.140388,0.0,1.0,0.241351,0.427902
7,2021-09-19,-0.384868,-0.139324,-0.287449,0.046528,1.0,1.0,1.000000,0.000000
8,2021-09-22,-0.501125,0.496563,-0.154346,0.161352,0.0,0.0,0.000000,0.000000
9,2021-09-24,-0.693721,1.000000,-0.140296,0.167533,0.0,1.0,0.001497,0.038662


In [21]:
ndvi_request(baguio,'2021-9-01', '2021-9-30','baguio')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-02,-1.000000,1.000000,0.362709,0.328503,0.0,1.0,0.672821,0.469183
1,2021-09-04,0.014223,0.153574,0.045114,0.017467,1.0,1.0,1.000000,0.000000
2,2021-09-07,-0.842520,1.000000,0.380785,0.312224,0.0,1.0,0.540253,0.498377
3,2021-09-09,-0.448567,1.000000,0.560301,0.254467,0.0,1.0,0.083001,0.275883
4,2021-09-12,0.015342,0.031300,0.024847,0.001861,1.0,1.0,1.000000,0.000000
5,2021-09-14,-1.000000,1.000000,0.361759,0.336002,0.0,1.0,0.592315,0.491404
6,2021-09-17,-0.181860,1.000000,0.560934,0.205673,0.0,1.0,0.241351,0.427902
7,2021-09-19,0.040269,0.326981,0.130907,0.062154,1.0,1.0,1.000000,0.000000
8,2021-09-22,-0.309771,1.000000,0.586798,0.239033,0.0,0.0,0.000000,0.000000
9,2021-09-24,-1.000000,1.000000,0.606467,0.256986,0.0,1.0,0.001497,0.038662


In [66]:
no_request(baguio,'2021-9-01', '2021-9-30','baguio')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-09-06,-0.000002,-0.000002,-0.000002,0.0,NaN,NaN,NaN,NaN
6,2021-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Cagayan de Oro City 

In [94]:
# ndbi
ndbi_request(cdo,'2021-9-01', '2021-9-30','cdo')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-03,-1.000000,1.000000,-0.230397,0.148182,0.0,1.0,0.093843,0.291610
1,2021-09-08,-0.577916,0.389879,-0.257706,0.075625,1.0,1.0,1.000000,0.000000
2,2021-09-13,-1.000000,1.000000,-0.225444,0.142896,0.0,1.0,0.115753,0.319929
3,2021-09-18,-0.494904,0.182573,-0.257207,0.067860,0.0,1.0,0.918403,0.273750
4,2021-09-23,-1.000000,1.000000,-0.199947,0.156101,0.0,1.0,0.339867,0.473664
5,2021-09-28,-0.475838,0.200000,-0.296681,0.078523,0.0,1.0,0.723266,0.447384


In [22]:
ndvi_request(cdo,'2021-9-01', '2021-9-30','cdo')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-03,-1.000000,1.000000,0.692368,0.293465,0.0,1.0,0.093843,0.291610
1,2021-09-08,-0.253896,0.818534,0.199881,0.208829,1.0,1.0,1.000000,0.000000
2,2021-09-13,-1.000000,1.000000,0.694019,0.273434,0.0,1.0,0.115753,0.319929
3,2021-09-18,-0.498380,0.963432,0.468509,0.167340,0.0,1.0,0.918403,0.273750
4,2021-09-23,-1.000000,1.000000,0.596778,0.323115,0.0,1.0,0.339867,0.473664
5,2021-09-28,-0.779404,1.000000,0.505680,0.238011,0.0,1.0,0.723266,0.447384


In [67]:
no_request(cdo,'2021-9-01', '2021-9-30','cdo')

,date,min,max,mean,stDev,min_clm,max_clm,mean_clm,stDev_clm
0,2021-09-01,-0.0,0.0,-0.0,0.0,NaN,NaN,NaN,NaN
1,2021-09-02,0.000001,0.000028,0.000015,0.000007,NaN,NaN,NaN,NaN
2,2021-09-03,-0.000008,0.000031,0.00001,0.000009,NaN,NaN,NaN,NaN
3,2021-09-04,0.000005,0.00002,0.00001,0.000004,NaN,NaN,NaN,NaN
4,2021-09-05,-0.000008,0.000024,0.000014,0.000008,NaN,NaN,NaN,NaN
5,2021-09-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-09-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-09-10,-0.000007,0.000026,0.000007,0.000009,NaN,NaN,NaN,NaN
